Alunos: Daniel de Paula e Gustavo Guerreiro

# Implementação do Algoritmo AES

O objetivo deste trabalho é a implementação do algoritmo de criptografia AES.

## Expansão de Chaves

In [578]:
def obter_matriz_de_estado(bloco):
    return [
        [bloco[0],  bloco[4],  bloco[8],  bloco[12]],
        [bloco[1],  bloco[5],  bloco[9],  bloco[13]],
        [bloco[2],  bloco[6],  bloco[10], bloco[14]],
        [bloco[3],  bloco[7],  bloco[11], bloco[15]]
    ]

In [579]:
def rot_word(palavra):
    return [palavra[1], palavra[2], palavra[3], palavra[0]]

In [580]:
def s_box():
    return [
        [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
        [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
        [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
        [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
        [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
        [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
        [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
        [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
        [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
        [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
        [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
        [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
        [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
        [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
        [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
        [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
    ]

Para consultar as tabelas S-Box, Tabela E e Tabela L, foram feitas manipulações usando operações bitwise (bitwise "e" e operador de deslocamento) a fim de isolar os 4 bits mais significativos e os 4 bits menos significativos.

In [581]:
def consultar_tabela(valor, tabela):
    x = (valor & 0b11110000) >> 4
    y = (valor & 0b00001111)
    return tabela[x][y]

def sub_word(palavra):
    nova_palavra = []
    for elemento in palavra:
        nova_palavra.append(consultar_tabela(elemento, s_box()))
    return nova_palavra

As round constants foram tratadas como um dicionário.

In [582]:
def gerar_round_constant(round):
    round_constants = {
        1:  [0x01, 0x00, 0x00, 0x00],
        2:  [0x02, 0x00, 0x00, 0x00],
        3:  [0x04, 0x00, 0x00, 0x00],
        4:  [0x08, 0x00, 0x00, 0x00],
        5:  [0x10, 0x00, 0x00, 0x00],
        6:  [0x20, 0x00, 0x00, 0x00],
        7:  [0x40, 0x00, 0x00, 0x00],
        8:  [0x80, 0x00, 0x00, 0x00],
        9:  [0x1b, 0x00, 0x00, 0x00],
        10: [0x36, 0x00, 0x00, 0x00],
    }
    return round_constants[round]

In [583]:
def aplicar_round_constant(palavra, numero_round):
    round_constant = gerar_round_constant(numero_round)
    resultado = []
    for elemento, constant in zip(palavra, round_constant):
        resultado.append(elemento ^ constant)
    return resultado

Foi utilizada a função zip para percorrer as duas palavras simultaneamente elemento a elemento.

In [584]:
def xor_palavras(palavra1, palavra2):
    resultado = []
    for elemento1, elemento2 in zip(palavra1, palavra2):
        resultado.append(elemento1 ^ elemento2)
    return resultado

Em diversos momentos precisamos consultar alguma coluna da matriz de estado ou da chave (palavra). Por isso a operação de transposição auxilia esse processo sendo possível tratar a coluna como uma lista.

In [585]:
def obter_coluna(matriz, indice):
    matriz_t = transpor_matriz(matriz)
    return matriz_t[indice].copy()

Implementação final da expansão da chave:

In [586]:
def expandir_round(chave_anterior, round):
    round_key = [[], [], [], []]
    palavra_inicial = obter_coluna(chave_anterior, -1)
    palavra_rotacionada = rot_word(palavra_inicial)
    palavra_substituida = sub_word(palavra_rotacionada)
    palavra_round_constant = aplicar_round_constant(palavra_substituida, round)
    palavra_com_anterior = xor_palavras(obter_coluna(chave_anterior, 0), palavra_round_constant)
    for indice, linha in enumerate(round_key):
        linha.append(palavra_com_anterior[indice])
    for i in range(1, 4):
        nova_palavra = xor_palavras(obter_coluna(round_key, i-1), obter_coluna(chave_anterior, i))
        for indice, linha in enumerate(round_key):
            linha.append(nova_palavra[indice])
    return round_key

def expandir_chaves(chave_inicial):
    round_keys = [chave_inicial]
    for i in range(1, 11):
        chave = round_keys[-1]
        round_keys.append(expandir_round(chave, i))
    return round_keys

## Cifragem

In [587]:
def add_round_key(texto_simples, round_key):
    resultado = []
    for coluna_matriz, coluna_chave in zip(texto_simples, round_key):
        palavra = []
        for elemento_matriz, elemento_chave in zip(coluna_matriz, coluna_chave):
            palavra.append(elemento_matriz ^ elemento_chave)
        resultado.append(palavra)
    return resultado

In [588]:
def sub_bytes(matriz):
    resultado = []
    for linha in matriz:
        resultado.append(sub_word(linha))
    return resultado

In [589]:
def shift_rows(matriz):
    return [
        [matriz[0][0], matriz[0][1], matriz[0][2], matriz[0][3]],
        [matriz[1][1], matriz[1][2], matriz[1][3], matriz[1][0]],
        [matriz[2][2], matriz[2][3], matriz[2][0], matriz[2][1]],
        [matriz[3][3], matriz[3][0], matriz[3][1], matriz[3][2]],
    ]

Constantes para as matrizes utilizadas na multiplicação de Galois

In [590]:
def matriz_multiplicacao():
    return [
        [2, 3, 1, 1],
        [1, 2, 3, 1],
        [1, 1, 2, 3],
        [3, 1, 1, 2]
    ]

def tabela_l():
    return [
        [0x00, 0x00, 0x19, 0x01, 0x32, 0x02, 0x1a, 0xc6, 0x4b, 0xc7, 0x1b, 0x68, 0x33, 0xee, 0xdf, 0x03],
        [0x64, 0x04, 0xe0, 0x0e, 0x34, 0x8d, 0x81, 0xef, 0x4c, 0x71, 0x08, 0xc8, 0xf8, 0x69, 0x1c, 0xc1],
        [0x7d, 0xc2, 0x1d, 0xb5, 0xf9, 0xb9, 0x27, 0x6a, 0x4d, 0xe4, 0xa6, 0x72, 0x9a, 0xc9, 0x09, 0x78],
        [0x65, 0x2f, 0x8a, 0x05, 0x21, 0x0f, 0xe1, 0x24, 0x12, 0xf0, 0x82, 0x45, 0x35, 0x93, 0xda, 0x8e],
        [0x96, 0x8f, 0xdb, 0xbd, 0x36, 0xd0, 0xce, 0x94, 0x13, 0x5c, 0xd2, 0xf1, 0x40, 0x46, 0x83, 0x38],
        [0x66, 0xdd, 0xfd, 0x30, 0xbf, 0x06, 0x8b, 0x62, 0xb3, 0x25, 0xe2, 0x98, 0x22, 0x88, 0x91, 0x10],
        [0x7e, 0x6e, 0x48, 0xc3, 0xa3, 0xb6, 0x1e, 0x42, 0x3a, 0x6b, 0x28, 0x54, 0xfa, 0x85, 0x3d, 0xba],
        [0x2b, 0x79, 0x0a, 0x15, 0x9b, 0x9f, 0x5e, 0xca, 0x4e, 0xd4, 0xac, 0xe5, 0xf3, 0x73, 0xa7, 0x57],
        [0xaf, 0x58, 0xa8, 0x50, 0xf4, 0xea, 0xd6, 0x74, 0x4f, 0xae, 0xe9, 0xd5, 0xe7, 0xe6, 0xad, 0xe8],
        [0x2c, 0xd7, 0x75, 0x7a, 0xeb, 0x16, 0x0b, 0xf5, 0x59, 0xcb, 0x5f, 0xb0, 0x9c, 0xa9, 0x51, 0xa0],
        [0x7f, 0x0c, 0xf6, 0x6f, 0x17, 0xc4, 0x49, 0xec, 0xd8, 0x43, 0x1f, 0x2d, 0xa4, 0x76, 0x7b, 0xb7],
        [0xcc, 0xbb, 0x3e, 0x5a, 0xfb, 0x60, 0xb1, 0x86, 0x3b, 0x52, 0xa1, 0x6c, 0xaa, 0x55, 0x29, 0x9d],
        [0x97, 0xb2, 0x87, 0x90, 0x61, 0xbe, 0xdc, 0xfc, 0xbc, 0x95, 0xcf, 0xcd, 0x37, 0x3f, 0x5b, 0xd1],
        [0x53, 0x39, 0x84, 0x3c, 0x41, 0xa2, 0x6d, 0x47, 0x14, 0x2a, 0x9e, 0x5d, 0x56, 0xf2, 0xd3, 0xab],
        [0x44, 0x11, 0x92, 0xd9, 0x23, 0x20, 0x2e, 0x89, 0xb4, 0x7c, 0xb8, 0x26, 0x77, 0x99, 0xe3, 0xa5],
        [0x67, 0x4a, 0xed, 0xde, 0xc5, 0x31, 0xfe, 0x18, 0x0d, 0x63, 0x8c, 0x80, 0xc0, 0xf7, 0x70, 0x07]
    ]

def tabela_e():
    return [
        [0x01, 0x03, 0x05, 0x0f, 0x11, 0x33, 0x55, 0xff, 0x1a, 0x2e, 0x72, 0x96, 0xa1, 0xf8, 0x13, 0x35],
        [0x5f, 0xe1, 0x38, 0x48, 0xd8, 0x73, 0x95, 0xa4, 0xf7, 0x02, 0x06, 0x0a, 0x1e, 0x22, 0x66, 0xaa],
        [0xe5, 0x34, 0x5c, 0xe4, 0x37, 0x59, 0xeb, 0x26, 0x6a, 0xbe, 0xd9, 0x70, 0x90, 0xab, 0xe6, 0x31],
        [0x53, 0xf5, 0x04, 0x0c, 0x14, 0x3c, 0x44, 0xcc, 0x4f, 0xd1, 0x68, 0xb8, 0xd3, 0x6e, 0xb2, 0xcd],
        [0x4c, 0xd4, 0x67, 0xa9, 0xe0, 0x3b, 0x4d, 0xd7, 0x62, 0xa6, 0xf1, 0x08, 0x18, 0x28, 0x78, 0x88],
        [0x83, 0x9e, 0xb9, 0xd0, 0x6b, 0xbd, 0xdc, 0x7f, 0x81, 0x98, 0xb3, 0xce, 0x49, 0xdb, 0x76, 0x9a],
        [0xb5, 0xc4, 0x57, 0xf9, 0x10, 0x30, 0x50, 0xf0, 0x0b, 0x1d, 0x27, 0x69, 0xbb, 0xd6, 0x61, 0xa3],
        [0xfe, 0x19, 0x2b, 0x7d, 0x87, 0x92, 0xad, 0xec, 0x2f, 0x71, 0x93, 0xae, 0xe9, 0x20, 0x60, 0xa0],
        [0xfb, 0x16, 0x3a, 0x4e, 0xd2, 0x6d, 0xb7, 0xc2, 0x5d, 0xe7, 0x32, 0x56, 0xfa, 0x15, 0x3f, 0x41],
        [0xc3, 0x5e, 0xe2, 0x3d, 0x47, 0xc9, 0x40, 0xc0, 0x5b, 0xed, 0x2c, 0x74, 0x9c, 0xbf, 0xda, 0x75],
        [0x9f, 0xba, 0xd5, 0x64, 0xac, 0xef, 0x2a, 0x7e, 0x82, 0x9d, 0xbc, 0xdf, 0x7a, 0x8e, 0x89, 0x80],
        [0x9b, 0xb6, 0xc1, 0x58, 0xe8, 0x23, 0x65, 0xaf, 0xea, 0x25, 0x6f, 0xb1, 0xc8, 0x43, 0xc5, 0x54],
        [0xfc, 0x1f, 0x21, 0x63, 0xa5, 0xf4, 0x07, 0x09, 0x1b, 0x2d, 0x77, 0x99, 0xb0, 0xcb, 0x46, 0xca],
        [0x45, 0xcf, 0x4a, 0xde, 0x79, 0x8b, 0x86, 0x91, 0xa8, 0xe3, 0x3e, 0x42, 0xc6, 0x51, 0xf3, 0x0e],
        [0x12, 0x36, 0x5a, 0xee, 0x29, 0x7b, 0x8d, 0x8c, 0x8f, 0x8a, 0x85, 0x94, 0xa7, 0xf2, 0x0d, 0x17],
        [0x39, 0x4b, 0xdd, 0x7c, 0x84, 0x97, 0xa2, 0xfd, 0x1c, 0x24, 0x6c, 0xb4, 0xc7, 0x52, 0xf6, 0x01]
    ]

In [591]:
def mul_galois(termo1, termo2):
    if termo1 == 0 or termo2 == 0:
        return 0
    if termo1 == 1:
        return termo2
    if termo2 == 1:
        return termo1
    numero1 = consultar_tabela(termo1, tabela_l())
    numero2 = consultar_tabela(termo2, tabela_l())
    soma = numero1 + numero2
    if soma > 0xff:
        soma = soma - 0xff
    resultado = consultar_tabela(soma, tabela_e())
    return resultado

In [592]:
def multiplicar_coluna(coluna, linha_mul):
    resultado = mul_galois(coluna[0], linha_mul[0])
    for i in range(1, len(linha_mul)):
        resultado ^= mul_galois(coluna[i], linha_mul[i])
    return resultado

In [593]:
def transpor_matriz(matriz):
    return [list(r) for r in zip(*matriz)]

In [594]:
def mix_columns(matriz):
    matriz = transpor_matriz(matriz)
    matriz_mul = matriz_multiplicacao()
    resultado = []
    for i in range(4):
        resultado.append([])
        for j in range(4):
            resultado[i].append(
                multiplicar_coluna(matriz[j], matriz_mul[i]))
    return resultado

Implementação final da cifragem AES:

In [595]:
def cifrar_aes(texto_simples, chave):
    texto_simples_matriz = obter_matriz_de_estado(texto_simples)
    round_keys = expandir_chaves(chave)
    a = add_round_key(texto_simples_matriz, round_keys[0])
    for i in range(1, 10):
        b = sub_bytes(a)
        c = shift_rows(b)
        d = mix_columns(c)
        a = add_round_key(d, round_keys[i])
    b = sub_bytes(a)
    c = shift_rows(b)
    e = add_round_key(c, round_keys[10])
    resultado = []
    for coluna in range(4):
        for linha in range(4):
            resultado.append(e[linha][coluna])
    return resultado

## Decifragem

In [596]:
def inv_shift_rows(matriz):
    return [
        [matriz[0][0], matriz[0][1], matriz[0][2], matriz[0][3]],
        [matriz[1][3], matriz[1][0], matriz[1][1], matriz[1][2]],
        [matriz[2][2], matriz[2][3], matriz[2][0], matriz[2][1]],
        [matriz[3][1], matriz[3][2], matriz[3][3], matriz[3][0]],
    ]

In [597]:
def inv_s_box():
    return [
        [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb],
        [0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb],
        [0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e],
        [0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25],
        [0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92],
        [0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84],
        [0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06],
        [0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b],
        [0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73],
        [0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e],
        [0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b],
        [0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4],
        [0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f],
        [0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef],
        [0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61],
        [0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]
    ]

In [598]:
def inv_sub_word(palavra):
    nova_palavra = []
    for elemento in palavra:
        nova_palavra.append(consultar_tabela(elemento, inv_s_box()))
    return nova_palavra

def inv_sub_bytes(matriz):
    resultado = []
    for linha in matriz:
        resultado.append(inv_sub_word(linha))
    return resultado

In [599]:
def inv_matriz_multiplicacao():
    return [
        [0x0e, 0x0b, 0x0d, 0x09],
        [0x09, 0x0e, 0x0b, 0x0d],
        [0x0d, 0x09, 0x0e, 0x0b],
        [0x0b, 0x0d, 0x09, 0x0e],
    ]

In [600]:
def inv_mix_columns(matriz):
    matriz = transpor_matriz(matriz)
    matriz_mul = inv_matriz_multiplicacao()
    resultado = []
    for i in range(4):
        resultado.append([])
        for j in range(4):
            resultado[i].append(
                multiplicar_coluna(matriz[j], matriz_mul[i]))
    return resultado

Implementação final da decifragem AES.

In [601]:
def decifrar_aes(texto_cifrado, chave):
    round_keys = expandir_chaves(chave)
    texto_cifrado_matriz = obter_matriz_de_estado(texto_cifrado)
    a = add_round_key(texto_cifrado_matriz, round_keys[10])
    b = inv_shift_rows(a)
    c = inv_sub_bytes(b)
    for i in range(9,0,-1):
        a = add_round_key(c, round_keys[i])
        d = inv_mix_columns(a)
        b = inv_shift_rows(d)
        c = inv_sub_bytes(b)
    a = add_round_key(c, chave)
    resultado = []
    for coluna in range(4):
        for linha in range(4):
            resultado.append(a[linha][coluna])

    return resultado

## Operações de preenchimento e despreenchimento do PKCS#7

In [602]:
def pkcs7(lista_bytes):
    lista_bytes = lista_bytes.copy()
    tamanho = len(lista_bytes)
    if tamanho % 16 == 0:
        lista_bytes.extend([16]*16)
        return lista_bytes
    blocos = [lista_bytes[i:i+16] for i in range(0, tamanho, 16)]
    ultimo_bloco = blocos[-1]
    diferenca = 16 - len(ultimo_bloco)
    lista_bytes.extend([diferenca]*diferenca)
    return lista_bytes

In [603]:
def remover_padding(lista_bytes):
    lista_bytes = lista_bytes.copy()
    ultimo_elemento = lista_bytes[-1]
    for i in range(ultimo_elemento):
        lista_bytes.pop()
    return lista_bytes

## Interação com o usuário

In [604]:
def processar_chave(texto_chave):
    chave_lista = texto_chave.split(',')
    chave_lista = [int(elemento) for elemento in chave_lista]
    if len(chave_lista) != 16:
        raise ValueError('Chave invalida. A chave precisa ter 16 dígitos.')
    return [chave_lista[i:i+4] for i in range(0, 16, 4)]

In [605]:
def ler_arquivo_binario():
    caminho = input('Insira o caminho para o arquivo.')
    while True:
        try:
            with open(caminho, 'rb') as arquivo:
                return list(arquivo.read())
        except FileNotFoundError:
            caminho = input('Caminho não encontrado. Insira o caminho para o arquivo novamente.')

def obter_chave():
    chave = input('Insira o chave de criptografia.')
    while True:
        try:
            return processar_chave(chave)
        except ValueError:
            chave = input('Chave incorreta. Insira o chave de criptografia novamente.')

def escrever_arquivo_binario(nome_arquivo, conteudo):
    with open(nome_arquivo, 'wb') as arquivo:
        arquivo.write(bytearray(conteudo))

def cifrar_arquivo(original, chave):
    print('Cifrando...')
    padding = pkcs7(original)
    blocos = [padding[i:i+16] for i in range(0, len(padding), 16)]
    cifra = cifrar_aes(blocos[0], chave)
    for i in range(1, len(blocos)):
        cifra.extend(cifrar_aes(blocos[i], chave))
    nome_arquivo = input('Digite o nome do arquivo de destino.')
    escrever_arquivo_binario(nome_arquivo, cifra)
    print('Cifra finalizada!')

def decifrar_arquivo(original, chave):
    print('Decifrando...')
    blocos = [original[i:i+16] for i in range(0, len(original), 16)]
    decifragem = decifrar_aes(blocos[0], chave)
    for i in range(1, len(blocos)):
        decifragem.extend(decifrar_aes(blocos[i], chave))
    decifragem = remover_padding(decifragem)
    nome_arquivo = input('Digite o nome do arquivo de destino.')
    escrever_arquivo_binario(nome_arquivo, decifragem)
    print('Decifragem finalizada!')

def aes():
    original = ler_arquivo_binario()
    chave = obter_chave()
    opcao = input('Digite 1 para cifrar ou 2 para decifrar: ').strip()

    match opcao:
        case '1':
            cifrar_arquivo(original, chave)
        case '2':
            decifrar_arquivo(original, chave)
        case _:
            print('Opção invalida!')

## Rodando programa

In [606]:
aes()

Decifrando...
Decifragem finalizada!
